In [1]:
''' 모듈화 '''

' 모듈화 '

In [2]:
from flask import Flask
from flask import request
from werkzeug.utils import secure_filename
import numpy as np
import tensorflow as tf
import pandas as pd
import joblib
import json
import uuid
import base64

# 필요한 PyTorch 라이브러리 불러오기
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image
from IPython.display import Image as display_image
import cv2

# 이미지 스타일 전이 관련

In [3]:
# 인코더(Encoder) 정의
vgg = nn.Sequential(
    nn.Conv2d(3, 3, (1, 1)),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(3, 64, (3, 3)),
    nn.ReLU(), # relu1-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 64, (3, 3)),
    nn.ReLU(), # relu1-2
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 128, (3, 3)),
    nn.ReLU(), # relu2-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 128, (3, 3)),
    nn.ReLU(), # relu2-2
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 256, (3, 3)),
    nn.ReLU(), # relu3-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(), # relu3-2
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(), # relu3-3
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(), # relu3-4
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 512, (3, 3)),
    nn.ReLU(), # relu4-1, this is the last layer used
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(), # relu4-2
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(), # relu4-3
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(), # relu4-4
    nn.MaxPool2d((2, 2), (2, 2), (0, 0), ceil_mode=True),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(), # relu5-1
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(), # relu5-2
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU(), # relu5-3
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 512, (3, 3)),
    nn.ReLU() # relu5-4
)

In [4]:
# 디코더(Decoder) 정의
decoder = nn.Sequential(
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(512, 256, (3, 3)),
    nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='nearest'),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 256, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(256, 128, (3, 3)),
    nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='nearest'),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 128, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(128, 64, (3, 3)),
    nn.ReLU(),
    nn.Upsample(scale_factor=2, mode='nearest'),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 64, (3, 3)),
    nn.ReLU(),
    nn.ReflectionPad2d((1, 1, 1, 1)),
    nn.Conv2d(64, 3, (3, 3)),
)

In [5]:
decoder.eval()
vgg.eval()

vgg_path = 'vgg_normalised.pth'
decoder_path = 'decoder.pth'

decoder.load_state_dict(torch.load(decoder_path))
vgg.load_state_dict(torch.load(vgg_path))

device = torch.device("cpu")
vgg.to(device)
decoder.to(device)

vgg = nn.Sequential(*list(vgg.children())[:31]) # ReLU4_1까지만 사용하기 위해 뒤쪽은 자름.

In [6]:
# 피처맵에 대해 평균과 표준편차를 구해서 리턴하는 함수
def calc_mean_std(feat, eps=1e-5):
    size = feat.size()
    assert (len(size) == 4)
    N, C = size[:2]
    feat_var = feat.view(N, C, -1).var(dim=2) + eps #하나의 벡터로 만든 뒤, variance를 구해주고, 0이 되지 않도록 작은 값을 더해줌
    feat_std = feat_var.sqrt().view(N, C, 1, 1)
    feat_mean = feat.view(N, C, -1).mean(dim=2).view(N, C, 1, 1)
    return feat_mean, feat_std

In [7]:
def adaptive_instance_normalization(content_feat, style_feat):
    assert (content_feat.size()[:2] == style_feat.size()[:2])
    size = content_feat.size()
    style_mean, style_std = calc_mean_std(style_feat)
    content_mean, content_std = calc_mean_std(content_feat)

    # 평균(mean)과 표준편차(std)를 이용하여 정규화 수행
    normalized_feat = (content_feat - content_mean.expand(size)) / content_std.expand(size)
    # 정규화 이후에 style feature의 statistics를 가지도록 설정
    return normalized_feat * style_std.expand(size) + style_mean.expand(size)

In [8]:
def style_transfer(vgg, decoder, content, style, alpha=1.0):
    assert (0.0 <= alpha <= 1.0)
    content_f = vgg(content)
    style_f = vgg(style)
    feat = adaptive_instance_normalization(content_f, style_f)
    feat = feat * alpha + content_f * (1 - alpha)
    return decoder(feat)

In [9]:
def test_transform(size=512):
    transform_list = []
    if size != 0:
        transform_list.append(transforms.Resize(size))
    transform_list.append(transforms.ToTensor())
    transform = transforms.Compose(transform_list)
    return transform

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 서버 실행

In [11]:
style_tf1 = test_transform()
style_tf2 = test_transform()
style_tf3 = test_transform()
style_tf4 = test_transform()

style1 = style_tf1(Image.open(str('style_image/style1.jpg')))
style2 = style_tf2(Image.open(str('style_image/style2.jpg')))
style3 = style_tf3(Image.open(str('style_image/style3.jpg')))
style4 = style_tf4(Image.open(str('style_image/style4.jfif')))

style1 = style1.to(device).unsqueeze(0)
style2 = style2.to(device).unsqueeze(0)
style3 = style3.to(device).unsqueeze(0)
style4 = style4.to(device).unsqueeze(0)

style__LS = [style1, style2, style3, style4]

In [12]:
# 하나의 스타일에 대해서만 적용.
def out_b64_string(secure_name, unique_name, style_num, content, style):
    save_style_transfer_image_path = 'save_style_transfer_image/'+ unique_name + secure_name + f'{style_num}.jpg'
    with torch.no_grad():
        output = style_transfer(vgg, decoder, content, style, alpha=0.5)
    output = output.cpu()
    
    save_image(output, save_style_transfer_image_path)
    img = cv2.imread(save_style_transfer_image_path)
    jpg_img = cv2.imencode('.jpg', img)
    
    b64_string = base64.b64encode(jpg_img[1]).decode('utf-8')
    
    return b64_string

In [ ]:
app = Flask(__name__)
# HTTP POST방식으로 전송된 이미지를 저장
@app.route('/styleimages4/', methods=['POST'])
def input_image1():
    content_tf = test_transform()
    #스타일 같은 것은 미리 함수밖에서 선언해서 끌고오기만 하기
    f = request.files['file']
    unique_name = str(uuid.uuid1())
    save_image_path = 'save_image/' + unique_name + secure_filename(f.filename)
    
    print(save_image_path)
    f.save(save_image_path)
    
    content = content_tf(Image.open(save_image_path))
    content = content.to(device).unsqueeze(0)
    
    b64_string__LS = []
    for idx, style_ in enumerate(style__LS):
        b64_string__LS.append(out_b64_string(secure_name, unique_name, idx, content, style_))
    
    hashtags_dict = {
                    'img1': b64_string__LS[0],
                    'img2': b64_string__LS[1],
                    'img3': b64_string__LS[2],
                    'img4': b64_string__LS[3],}
    hashtags_json = json.dumps(hashtags_dict, sort_keys=True)
    return hashtags_json

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=60001)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:60001
 * Running on http://172.30.1.46:60001 (Press CTRL+C to quit)


save_image/35801b64-ccff-11ec-884c-c03c596ba129jpg


C:\Users\User\anaconda3\envs\hashtaig\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
127.0.0.1 - - [06/May/2022 14:42:08] "POST /styleimages4/ HTTP/1.1" 200 -


save_image/4ac8dcec-ccff-11ec-8209-c03c596ba129jpg


127.0.0.1 - - [06/May/2022 14:42:42] "POST /styleimages4/ HTTP/1.1" 200 -


save_image/740f1428-ccff-11ec-9d32-c03c596ba129jpg


127.0.0.1 - - [06/May/2022 14:43:52] "POST /styleimages4/ HTTP/1.1" 200 -
